In [3]:
from hig_data.coco import COCOStuffDataset, CocoStuffGraphDataset
import torch
base = '/home/rfsm2/rds/hpc-work/'
img_path = base+'coco_val2017_256-sd.zip'
mask_path = base+'coco_val2017_masks_256.zip'
coco_graph = CocoStuffGraphDataset(img_path, mask_path, latent_images=True)
data = coco_graph[4]
data



RelaxedHeteroData(
  image=[1, 8, 32, 32],
  mask=[1, 1, 256, 256],
  image_node={
    x=[1024, 1],
    pos=[1024, 2],
  },
  class_node={
    x=[3, 183],
    pos=[3, 2],
  },
  (class_node, class_edge, class_node)={ edge_index=[2, 6] },
  (class_node, class_to_image, image_node)={ edge_index=[2, 971] }
)

In [4]:
from training.networks_edm2_hignn import Precond
precond = Precond(32, 3, gnn_metadata = data.metadata())

In [5]:
import torch
images = torch.randn(1, 3, 32, 32)
rnd_normal = torch.randn([images.shape[0], 1, 1, 1], device=images.device)
sigma = (rnd_normal).exp()

data = coco_graph[1]
out = precond(images, sigma, data)
out.shape

torch.Size([1, 3, 32, 32])

In [8]:
from training.training_loop_hignn import training_loop
import torch
from torch_utils import distributed as dist
torch.multiprocessing.set_start_method('spawn')
dist.init()
dist.print0('Setting up training config...')
training_loop(dataset_kwargs=dict(class_name='hig_data.coco.CocoStuffGraphDataset',
image_path=img_path, mask_path=mask_path, latent_images=True),)

Setting up training config...
Loading dataset...
Setting up encoder...
ref_image torch.Size([1, 8, 32, 32])
Constructing network...

Precond                Parameters  Buffers  Output shape        Datatype
---                    ---         ---      ---                 ---     
unet.emb_fourier       -           384      [256, 192]          float32 
unet.emb_noise         147456      -        [256, 768]          float32 
unet.enc.32x32_conv    8640        -        [256, 192, 32, 32]  float16 
unet.enc.32x32_block0  811009      -        [256, 192, 32, 32]  float16 
unet.enc.32x32_block1  811009      -        [256, 192, 32, 32]  float16 
unet.enc.32x32_block2  811009      -        [256, 192, 32, 32]  float16 
unet.enc.16x16_down    811009      -        [256, 192, 16, 16]  float16 
unet.enc.16x16_block0  3022849     -        [256, 384, 16, 16]  float16 
unet.enc.16x16_block1  2949121     -        [256, 384, 16, 16]  float16 
unet.enc.16x16_block2  2949121     -        [256, 384, 16, 16]  

TypeError: cannot pickle 'BufferedReader' instances